<a href="https://colab.research.google.com/github/aishwary247/ML/blob/main/NaiveBayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('/content/Corona_train.csv')
df.head()

,ID,Sentiment,CoronaTweet
0,22979,Positive,I see all kinds of academics already whipping ...
1,9880,Negative,@HenrySmithUK can you raise with Boris please ...
2,35761,Negative,It s a confusing odd time for the shopping pub...
3,37968,Positive,Blog Summary: The Impact of COVID-19 on the Ca...
4,19709,Neutral,??????? ??????? ???\r\r\nWaiting in a long Que...


In [3]:
# Removing URL's
def remove_url(text):
  pattern = re.compile(r'https?://\S+|www\.\S+')
  return pattern.sub(r'',text)

In [4]:
import re
# Define a function for text cleaning
def clean_text(text):
    if pd.notnull(text) and isinstance(text, str):
        text = text.lower()
        text = re.sub('<br />', '', text)
        text = re.sub('\n', '', text)
        text = re.sub(r'[^\w\s]', '', text)
    return text



In [5]:
df["CoronaTweet"]= df["CoronaTweet"].apply(remove_url)
df["CoronaTweet"]= df["CoronaTweet"].apply(clean_text)

In [6]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [7]:
def stem(text):
  y=[]
  for i in text.split() :
    y.append(ps.stem(i))

  return " ".join(y)

In [8]:
df["CoronaTweet"]= df["CoronaTweet"].apply(stem)

In [9]:
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
stwords = stopwords.words('english')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = nltk.word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

In [11]:
df["CoronaTweet"]= df["CoronaTweet"].apply(remove_stopwords)

In [12]:
!pip install unidecode


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 3.8 MB/s eta 0:00:00


In [13]:
from unidecode import unidecode

def strip_accents(text):
    return unidecode(text)

In [14]:
df["CoronaTweet"]= df["CoronaTweet"].apply(strip_accents)

In [15]:
# Function to remove words with numbers, excluding cases like "covid19"
def remove_numerical_words(text):
    return ' '.join(word for word in text.split() if not (re.search(r'\d', word) and not re.search(r'[\W_]*\d*covid[\W_]*\d*', word, flags=re.IGNORECASE)))

def replace_covid_words(text):
    return re.sub(r'[\W_]*\d*covid[\W_]*\d*', 'covid', text, flags=re.IGNORECASE)



In [16]:
df['CoronaTweet']= df['CoronaTweet'].apply(remove_numerical_words)
df['CoronaTweet']= df['CoronaTweet'].apply(replace_covid_words)
df

,ID,Sentiment,CoronaTweet
0,22979,Positive,see kind academ alreadi whipcovid relat projec...
1,9880,Negative,henrysmithuk rais bori pleas supermarket contr...
2,35761,Negative,confus odd time shop public store close flight...
3,37968,Positive,blog summari impactcovid canadian residenti ho...
4,19709,Neutral,wait long queue enter supermarket final saw en...
...,...,...,...
37859,20253,Neutral,doubl tap rememb employ follow walletgyd tip l...
37860,38926,Positive,struggl understand whi supermarket food shop a...
37861,35889,Positive,anyon interest checklist consum impliccovidpol...
37862,40641,Positive,agre hypothesi indian societi might chang fore...


In [17]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 40000 ,stop_words='english')

In [18]:
vector = cv.fit_transform(df['CoronaTweet']).toarray()
vector.shape

(37864, 40000)

In [ ]:
arr = cv.get_feature_names_out()
print(arr[1:1000])

In [20]:
df.shape[0]

37864

In [21]:
df.Sentiment.unique()

array(['Positive', 'Negative', 'Neutral'], dtype=object)

In [22]:
# m = df.shape[0]
# y = np.zeros((m))
# for i in range(m):
#   if df["Sentiment"][i] == "Positive" :
#     y[i] = 0
#   elif df["Sentiment"][i] == "Neutral" :
#     y[i] = 1
#   else :
#     y[i] = 2

# print(y)
sentiment_mapping = {"Positive": 0, "Neutral": 1, "Negative": 2}
y = np.array([sentiment_mapping[s] for s in df["Sentiment"]])
print(y)

[0 2 2 ... 0 0 1]


In [23]:
y[:10]

array([0, 2, 2, 0, 1, 1, 0, 0, 0, 0])

In [24]:
vector.shape, y.shape

((37864, 40000), (37864,))

In [25]:
# vector, y
m = 37864

sigmay0 = 0
sigmay1 = 0
sigmay2 = 0
for ele in y :
  if ele == 0:
    sigmay0 += 1
  elif ele == 1:
    sigmay1 += 1
  else :
    sigmay2 += 1


In [26]:
sigmay0+sigmay1+sigmay2

37864

In [27]:
m, n = vector.shape
phi0 = np.zeros((n))
phi1 = np.zeros((n))
phi2 = np.zeros((n))


# for k in range(n):
#   num0 = 0
#   num1 = 0
#   num2 = 0

#   for i in range(m):
#     if vector[i][k] == 1 and y[i] == 0 :
#       num0 += 1
#     elif vector[i][k] == 1 and y[i] == 1 :
#       num1 += 1
#     elif vector[i][k] == 1 and y[i] == 2 :
#       num2 += 1
#   phi0[k] =  (num0 + 1)/(sigmay0+3)
#   phi1[k] =  (num1 + 1)/(sigmay1+3)
#   phi2[k] =  (num2 + 1)/(sigmay2+3)



for k in range(n):
    num0 = np.sum((vector[:, k] == 1) & (y == 0))
    num1 = np.sum((vector[:, k] == 1) & (y == 1))
    num2 = np.sum((vector[:, k] == 1) & (y == 2))

    phi0[k] = (num0 + 1) / (sigmay0 + 3)
    phi1[k] = (num1 + 1) / (sigmay1 + 3)
    phi2[k] = (num2 + 1) / (sigmay2 + 3)

In [28]:
phiy = np.array((sigmay0/m, sigmay1/m, sigmay2/m))
phiy

array([0.43846398, 0.18740756, 0.37412846])

In [29]:
# Baic implementation with loops not time efficient
#def sigmap( X , y ):
#   if(y==0):
#     pdt = 1
#     for i in range(X.shape[0]):
#       if X[i] == 0 :
#         pdt *=(1-phi0[i])
#       else :
#         pdt *= phi0[i]
#     return pdt

#   elif(y==1):
#     pdt = 1
#     for i in range(X.shape[0]):
#       if X[i] == 0 :
#         pdt *=(1-phi1[i])
#       else :
#         pdt *= phi1[i]
#     return pdt

#   else:
#     pdt = 1
#     for i in range(X.shape[0]):
#       if X[i] == 0 :
#         pdt *=(1-phi2[i])
#       else :
#         pdt *= phi2[i]
#     return pdt

# def probab( y , X):
#   return (sigmap(X,y)*phiy[y])/(sigmap(X,0)*phiy[0]+sigmap(X,1)*phiy[1]+sigmap(X,2)*phiy[2])

# def predict(X):
#   probabilities = [probab(0, X), probab(1, X), probab(2, X)]

#   if max(probabilities) == probabilities[0]:
#       return 0
#   elif max(probabilities) == probabilities[1]:
#       return 1
#   else:
#        return 2



In [30]:
# import time
# start = time.time()
# print(predict(vector[0]))
# time1 = time.time()-start
# print(time1*30000)

In [31]:
# using Numpy array operations to improve time efficiency
def sigmap(X, phi):
    return np.prod(np.where(X == 0, 1 - phi, phi))

def probab(y, X, phi, phiy):
    numerator = sigmap(X, phi[y]) * phiy[y]
    denominator = np.sum([sigmap(X, phi[i]) * phiy[i] for i in range(3)])
    return numerator / denominator

def predict(X, phi, phiy):
    probabilities = [probab(i, X, phi, phiy) for i in range(3)]
    return np.argmax(probabilities)



In [32]:
phi = [phi0,phi1,phi2]

In [33]:
import time
start = time.time()
print(predict(vector[0],phi,phiy))
time1 = time.time()-start
print(time1*30000)

0
163.47885131835938


In [34]:
y_pred = np.zeros(m)
for i in range(m):
  y_pred[i] = predict(vector[i],phi,phiy)

In [35]:
y_pred

array([0., 2., 2., ..., 0., 1., 0.])

In [36]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
accuracy = accuracy_score(y, y_pred)

print(f'Accuracy: {accuracy:.4f}')

# Confusion matrix
conf_matrix = confusion_matrix(y, y_pred)
print('\nConfusion Matrix:')
print(conf_matrix)

# Classification report
class_report = classification_report(y, y_pred)
print('\nClassification Report:')
print(class_report)



Accuracy: 0.8015

Confusion Matrix:
[[14725   280  1597]
 [ 2035  3794  1267]
 [ 2052   285 11829]]

Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.89      0.83     16602
           1       0.87      0.53      0.66      7096
           2       0.81      0.84      0.82     14166

    accuracy                           0.80     37864
   macro avg       0.82      0.75      0.77     37864
weighted avg       0.81      0.80      0.80     37864

